## A Real-Time Map Visualization of COVID-19

This will be a short tutorial of how to present a dynamic map visualization primarily using folium and pandas. The temporal feature of this dataset spans from late January to early March. 

## Final Output

<img src="./china_covid-19_visualization_confirmed.gif">
<img src="./china_covid-19_visualization_recovered.gif">

### Credits
- [Here's the source of the data](https://www.kaggle.com/sudalairajkumar/novel-corona-virus-2019-dataset/data)
- [GeoJson file for China](https://github.com/yezongyang/china-geojson)

### A Detailed Version [http://andy971022.com/58e7c203-b961-4cdc-b242-6b61c5eb59a8/](http://andy971022.com/58e7c203-b961-4cdc-b242-6b61c5eb59a8/)
### My Personal Blog : [http://andy971022.com/](http://andy971022.com/)

In [ ]:
cp -r ../input/covid-19-china-visualization/. .

In [ ]:
import pandas as pd
import pkg_resources
# pkg_resources.require("folium==0.8.3")
import folium
from folium import Map, LayerControl, Choropleth

In [ ]:
CSV_PATH = "./CSV_Files/"
CSV_FILE = "covid_19_data.csv"
GEOJSON_PATH = "./GeoJson/"
GEOJSON_FILE = "china.json"
FILTER = ["Mainland China","Hong Kong", "Macau", "Taiwan"]

df = pd.read_csv(CSV_PATH + CSV_FILE)

In [ ]:
## Filter out Mainland China, Hong Kong, Macau, and Taiwan
df = df[df["Country/Region"].isin(FILTER)]
df

In [ ]:
## Assign the corresponding geojson id to values in Province/State so that we can draw a map for them
import json
geojson = json.load(open(GEOJSON_PATH + GEOJSON_FILE))
for values in geojson["features"]:
    print(values["properties"])

In [ ]:
## Gotta correspond their English names to that of Chinese MANUALLY
print(df["Province/State"].unique())
print(len(df["Province/State"].unique()))

In [ ]:
## In alphabetical order
ids = ['34','11','50','35','62','44','45','52','46','13','23','41','81','42','43','15','32','36','22','21','82','64','63','61','37','31','14','51','71','12','54','65','53','33']

## Lets check if we have mistaken the number or not

print(len(set(ids))) ## At least the number is right

In [ ]:
## Let's make a small dataframe so that we can do a left join
province_df = pd.DataFrame({"Province/State":df["Province/State"].unique(), "id":ids})
province_df

In [ ]:
## Let's do a left join!

df = df.merge(province_df,on = "Province/State", how = "left")

In [ ]:
df

In [ ]:
## Let's draw a map for a single day!
m = Map(location = [35,110],
        zoom_start = 4)
Choropleth(geo_data = GEOJSON_PATH + GEOJSON_FILE,
            name = 'choropleth',
            data = df[df["ObservationDate"] == "01/22/2020"],
            columns = ['id','Confirmed'],
            key_on = 'feature.properties.id',
            fill_opacity = 0.7,
            line_opacity = 0.2,
            line_color = 'red',
            fill_color = 'YlOrRd'
          ).add_to(m)
LayerControl().add_to(m)
m


In [ ]:
## Let's write the above code into a function so that we can automate the generation of maps
## http://github.com/python-visualization/folium/issues/220 sometimes error happens
def plot_map(date):
    m = Map(location = [35,110],
        zoom_start = 4)
    Choropleth(geo_data = GEOJSON_PATH + GEOJSON_FILE,
                name = 'choropleth',
                data = df[df["ObservationDate"] == date],
                columns = ['id','Confirmed'],
                key_on = 'feature.properties.id',
                fill_opacity = 0.7,
                line_opacity = 0.2,
                line_color = 'red',
                fill_color = 'YlOrRd'
              ).add_to(m)
    LayerControl().add_to(m)
    return m

In [ ]:
## Let's loop through the dates and plot them
print(df.ObservationDate.unique())

In [ ]:
from IPython.core.interactiveshell import InteractiveShell 
InteractiveShell.ast_node_interactivity = "all"

## Tibet seems to be missing out in some of the first few days
## Hubei is distorting our scale -- maybe we need a custom scale

for date in df.ObservationDate.unique()[:5]:
    maps = plot_map(date)
    maps

In [ ]:
## Let's modify the previous function again with our custom color scale

## You can also define your scale
# def my_scale(column,m):
#     import branca
#     index = [column.quantile(q=i/10) for i in range(0,12,2)]
#     colormap = branca.colormap.linear.YlOrRd_09.scale(column.min(), column.max())
#     colormap = colormap.to_step(index=index)
#     colormap.caption = 'Confirmed Numbers'
#     colormap.add_to(m)

## fill_color (string, default 'blue') – Area fill color.
## Can pass a hex code, color name, or if you are binding data, 
## one of the following color brewer palettes: ‘BuGn’, ‘BuPu’, ‘GnBu’, ‘OrRd’,
## ‘PuBu’, ‘PuBuGn’, ‘PuRd’, ‘RdPu’, ‘YlGn’, ‘YlGnBu’, ‘YlOrBr’, and ‘YlOrRd’.

def plot_map(date,column,line_color,fill_color):
    maximum = df[df["ObservationDate"] == date][column].max()
    maximum = maximum if maximum > 10000 else 10000
    m = Map(location = [35,100],
        zoom_start = 4)
    Choropleth(geo_data = GEOJSON_PATH + GEOJSON_FILE,
                name = 'choropleth',
                data = df[df["ObservationDate"] == date],
                columns = ['id', column],
                key_on = 'feature.properties.id',
                fill_opacity = 0.7,
                line_opacity = 0.2,
                line_color = line_color,
                fill_color = fill_color,
#                 bins = [df[df["ObservationDate"] == date]["Confirmed"].quantile(q=(10*i)**(0.5)/10) for i in range(0,12,2)]
                bins = [0,50,200,500,1000,maximum], ## as long as it's reasonable
              ).add_to(m)
#     my_scale(df[df["ObservationDate"] == date]["Confirmed"],m)
    LayerControl().add_to(m)
    m.save(f"./maps/{column}/{date.replace('/','')}.html")
    return m


In [ ]:
## https://github.com/python-visualization/folium/issues/35 saving html as images

for date in df.ObservationDate.unique():
    maps = plot_map(date,'Confirmed','red','YlOrRd')
    maps
    maps = plot_map(date,'Recovered','green','BuGn') ## Though we should be visualizing recovery rate instead
    maps

In [ ]:
# Let's screenshot our html files and make it into a gif

# !apt update -y
# !apt install cutycapt -y ## This is capable of screenshoting our html files. Install this if you don't have it
# !for file in ./maps/Confirmed/*.html; do cutycapt --url="${file%.*}".html --out="${file%.*}".png --delay=1000 ; echo "${file%.*}"; done

# import os
# import subprocess

# ## We're just open html files and taking screenshots via cutycapt

# folder =  "Confirmed"
# for file in os.listdir(f"./maps/{folder}"):
#     command = f"cutycapt --url=file://{os.getcwd()}/maps/{folder}/{file} --out=./images/{folder}/{file.split('.')[0]}.png --delay=1000"
#     subprocess.run(command.split(" "))
# #     print(command)



In [ ]:
# folder =  "Recovered"
# for file in os.listdir(f"./maps/{folder}"):
#     command = f"cutycapt --url=file://{os.getcwd()}/maps/{folder}/{file} --out=./images/{folder}/{file.split('.')[0]}.png --delay=1000"
#     subprocess.run(command.split(" "))

In [ ]:
## Couldn't apt get the tool I wanted, so I ran this on local and imported directly
## Run The above code if you are able to get cutycapt running on local
!cp -r ../input/covid-19-china-images/images ./

In [ ]:
## Example image
from IPython.display import Image
Image(filename='./images/Confirmed/03092020.png') 

In [ ]:
## Making gif out of the images
## run this if you're on local

# import os
# import subprocess

# command = "convert -delay 10 -loop 0 ./images/Recovered/*.png ./china_covid-19_visualization_recovered.gif"
# subprocess.run(command.split(" "))
# command = "convert -delay 10 -loop 0 ./images/Confirmed/*.png ./china_covid-19_visualization_confirmed.gif"
# subprocess.run(command.split(" "))




In [ ]:
from PIL import Image
import glob

files = sorted(glob.glob('./images/Confirmed/*.png'))
images = list(map(lambda file: Image.open(file), files))
images[0].save('china_covid-19_visualization_confirmed.gif', save_all=True, append_images=images[1:], duration=10, loop=0)

files = sorted(glob.glob('./images/Recovered/*.png'))
images = list(map(lambda file: Image.open(file), files))
images[0].save('china_covid-19_visualization_recovered.gif', save_all=True, append_images=images[1:], duration=10, loop=0)

In [ ]:
## Example gif
## run this if your on local
# from IPython.display import HTML
# HTML('<img src="./china_covid-19_visualization_confirmed.gif">')

<img src="./china_covid-19_visualization_confirmed.gif">
<img src="./china_covid-19_visualization_recovered.gif">